# KLASIFIKASI GAMBAR BURUNG

#### Kelompok 2 B2 2020 
<li> dataset : 

In [ ]:
import os 
import shutil 
import re
import random 
import tensorflow 
import keras  
import glob 
from skimage import io
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline

import tensorflow as tf
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
root_path = r'D:\SEMESTER 5\KB_2_B2\dataset'
source_path = os.path.join(root_path,'data gambar') #tempat dataset disimpan
destination_path = os.path.join(root_path, 'data gambar fix')
class_aves =['Order Anseriformes','Order Apodiformes', 'Order Apterygiformes']

In [ ]:
#melihat banyak data gambar dlm folder
folders = {}

for folder in os.listdir(source_path):
    folders[folder] = len(os.listdir(os.path.join(source_path,  folder)))

folders

In [ ]:
if os.path.isdir(os.path.join(destination_path, 'data_training')):
    print(f"path {os.path.join(destination_path, 'data_training')} sudah ada")
else:
    os.makedirs(os.path.join(destination_path, 'data_training'))
    for folder in folders.keys():
        os.makedirs(os.path.join(destination_path, f'data_training/{folder}'))

if os.path.isdir(os.path.join(destination_path, 'data_testing')):
    print(f"path {os.path.join(destination_path, 'data_testing')} sudah ada")
else:
    os.makedirs(os.path.join(destination_path, 'data_testing'))
    for folder in folders.keys():
        os.makedirs(os.path.join(destination_path, f'data_testing/{folder}'))

In [ ]:
aves_path = os.path.join(source_path,class_aves[1],'*')
aves_path = glob.glob(aves_path)

In [ ]:
# #Menampilkan gambar dari dataset
# image = io.imread(aves_path[2])  

# #memplot gambar original
# i, (im1) = plt.subplots(1)
# i.set_figwidth(15)
# im1.imshow(image)

In [ ]:
files_name = {
     'Order Anseriformes': [],
     'Order Apodiformes': [],
     'Order Apterygiformes': [],
     'Order Caprimulgiformes': [],
     'Order Casuariiformes': [],
     'Order Charadriiformes': [],
     'Order Ciconiiformes': [],
     'Order Coliiformes': [],
     'Order Columbiformes': [],
     'Order Coraciiformes': [],
     'Order Cuculiformes': [],
     'Order Falconiformes': [],
     'Order Galliformes': [],
     'Order Gaviiformes': [],
     'Order Gruiformes': [],
     'Order Passeriformes': [],
     'Order Pelecaniformes': [],
     'Order Phoenicopteriformes': [],
     'Order Piciformes': [],
     'Order Podicipediformes': [],
     'Order Procellariiformes': [],
     'Order Psittaciformes': [],
     'Order Pteroclidiformes': [],
     'Order Rheiformes': [],
     'Order Sphenisciformes': [],
     'Order Strigiformes': [],
     'Order Struthioniformes': [],
     'Order Tinamiformes': [],
     'Order Trogoniformes': [],
     'Order Turniciformes': [],
}

files_train = {
        'Order Anseriformes': [],
     'Order Apodiformes': [],
     'Order Apterygiformes': [],
     'Order Caprimulgiformes': [],
     'Order Casuariiformes': [],
     'Order Charadriiformes': [],
     'Order Ciconiiformes': [],
     'Order Coliiformes': [],
     'Order Columbiformes': [],
     'Order Coraciiformes': [],
     'Order Cuculiformes': [],
     'Order Falconiformes': [],
     'Order Galliformes': [],
     'Order Gaviiformes': [],
     'Order Gruiformes': [],
     'Order Passeriformes': [],
     'Order Pelecaniformes': [],
     'Order Phoenicopteriformes': [],
     'Order Piciformes': [],
     'Order Podicipediformes': [],
     'Order Procellariiformes': [],
     'Order Psittaciformes': [],
     'Order Pteroclidiformes': [],
     'Order Rheiformes': [],
     'Order Sphenisciformes': [],
     'Order Strigiformes': [],
     'Order Struthioniformes': [],
     'Order Tinamiformes': [],
     'Order Trogoniformes': [],
     'Order Turniciformes': [],
}

files_test = {
     'Order Anseriformes': [],
     'Order Apodiformes': [],
     'Order Apterygiformes': [],
     'Order Caprimulgiformes': [],
     'Order Casuariiformes': [],
     'Order Charadriiformes': [],
     'Order Ciconiiformes': [],
     'Order Coliiformes': [],
     'Order Columbiformes': [],
     'Order Coraciiformes': [],
     'Order Cuculiformes': [],
     'Order Falconiformes': [],
     'Order Galliformes': [],
     'Order Gaviiformes': [],
     'Order Gruiformes': [],
     'Order Passeriformes': [],
     'Order Pelecaniformes': [],
     'Order Phoenicopteriformes': [],
     'Order Piciformes': [],
     'Order Podicipediformes': [],
     'Order Procellariiformes': [],
     'Order Psittaciformes': [],
     'Order Pteroclidiformes': [],
     'Order Rheiformes': [],
     'Order Sphenisciformes': [],
     'Order Strigiformes': [],
     'Order Struthioniformes': [],
     'Order Tinamiformes': [],
     'Order Trogoniformes': [],
     'Order Turniciformes': [],
}

In [ ]:
for type in files_name.keys():
    for file in os.listdir(os.path.join(source_path, type)):
        if os.path.getsize(os.path.join(source_path, f'{type}/{file}')):
            files_name[type].append(file)
        else:
            print(f'{file} dilewatkan')

In [ ]:
#test melihat panjang sebuah folder 
len(files_name['Order Turniciformes'])

In [ ]:
split_point = {
     'Order Anseriformes': 0,
     'Order Apodiformes': 0,
     'Order Apterygiformes': 0,
     'Order Caprimulgiformes': 0,
     'Order Casuariiformes': 0,
     'Order Charadriiformes': 0,
     'Order Ciconiiformes': 0,
     'Order Coliiformes': 0,
     'Order Columbiformes': 0,
     'Order Coraciiformes': 0,
     'Order Cuculiformes': 0,
     'Order Falconiformes': 0,
     'Order Galliformes': 0,
     'Order Gaviiformes': 0,
     'Order Gruiformes': 0,
     'Order Passeriformes': 0,
     'Order Pelecaniformes': 0,
     'Order Phoenicopteriformes': 0,
     'Order Piciformes': 0,
     'Order Podicipediformes': 0,
     'Order Procellariiformes': 0,
     'Order Psittaciformes': 0,
     'Order Pteroclidiformes': 0,
     'Order Rheiformes': 0,
     'Order Sphenisciformes': 0,
     'Order Strigiformes': 0,
     'Order Struthioniformes': 0,
     'Order Tinamiformes': 0,
     'Order Trogoniformes': 0,
     'Order Turniciformes': 0,
}

for key in split_point.keys():
    split_point[key] = int(len(files_name[key])*.8)

for key in files_train.keys():
    files_train[key].append(files_name[key][:split_point[key]])
    files_test[key].append(files_name[key][split_point[key]:])


In [ ]:
len(files_train['Order Anseriformes'][0])

In [ ]:
len(files_test['Order Anseriformes'][0])

In [ ]:
source_path

In [ ]:
destination_path

In [ ]:
for type in files_name.keys():
    for file in files_train[type][0]:
        shutil.copyfile(
            os.path.join(source_path, f'{type}/{file}'),
            os.path.join(destination_path, f'data_training/{type}/{file}')
        )
    for file in files_test[type][0]:
        shutil.copyfile(
            os.path.join(source_path, f'{type}/{file}'),
            os.path.join(destination_path, f'data_testing/{type}/{file}')
        )